In [ ]:
import pandas as pd

trx_df = pd.read_csv(
    "data/Transactions.csv",
    encoding="utf-8-sig",
    sep=",",
    usecols=["Date", "Heure", "Produit", "Code ISIN", "Quantité", "Montant négocié"],
)

trx_df.head()

In [ ]:
portfolio_df = pd.read_csv(
    "data/Portfolio.csv",
    encoding="utf-8-sig",
    sep=",",
    usecols=["Ticker/ISIN", "Clôture", "Devise", "Montant en EUR"],
)

portfolio_df["Devise"] = portfolio_df["Devise"].str.split(" ").str[1]

# NA are not values that we need, let's drop them
portfolio_df = portfolio_df.dropna()

# Convert to float
portfolio_df["Clôture"] = portfolio_df["Clôture"].str.replace(",", ".", regex=False)
portfolio_df["Clôture"] = portfolio_df["Clôture"].astype(float)
portfolio_df["Montant en EUR"] = portfolio_df["Montant en EUR"].str.replace(",", ".", regex=False)
portfolio_df["Montant en EUR"] = portfolio_df["Montant en EUR"].astype(float)
portfolio_df["Devise"] = portfolio_df["Devise"].astype(float)

portfolio_df.head()

In [ ]:
# Should be either A for EUR to EUR, or the same everywhere for USD to EUR as same time for all (so same rate)
portfolio_df["fx_rate"] = portfolio_df["Montant en EUR"] / portfolio_df["Devise"]
portfolio_df.head()

In [ ]:
# We don't need these columns anymore as fx_rate as been calculated
portfolio_df.drop(columns=["Montant en EUR", "Devise"], inplace=True)


df = pd.merge(trx_df, portfolio_df, left_on="Code ISIN", right_on="Ticker/ISIN", how="left")
df.drop(columns=["Ticker/ISIN"], inplace=True)

df.head()

In [ ]:
lines = df.groupby(["Produit", "Code ISIN"]).agg({
    "Quantité": "sum",
    "Montant négocié": "sum",
    "Clôture": "first",
    "fx_rate": "first"
})

lines

In [ ]:
# Clôture is NaN when all positions have been sold.
# Montant négocié is the gains/losses for this asset.
# When Clôture is not NaN, we still have all/some positions for this asset.
# Need to compute the gains/losses based on the market price (Clôture).
# Montant négocié represents what we paid for this asset, and might include some gains/losses already if partially sold.

lines.fillna({"Clôture": 0, "fx_rate": 0 }, inplace=True)
lines["Gains/Losses"] = lines["Montant négocié"] + (lines["Quantité"] * lines["Clôture"] * lines["fx_rate"])

lines

In [ ]:
# Don't have the same result as in Degiro UI because of rounding - less precision in the CSV file
lines["Gains/Losses"].sum()